# CS 224N - CBT Adversarial Dataset Evaluation
Evaluating our methods on an adversarial dataset of CBT examples.

In [1]:
import torch
print("Using GPU: " + str(torch.cuda.is_available()))

Using GPU: True


## Load in 1:1 Real to Fake Words
Load in from the `.csv` file.

In [6]:
import pandas as pd

# Import CSV and get the real to fake
rtf_df = pd.read_csv("datasets/realtofake.csv")
real_words_list = rtf_df["Real"].tolist()
fake_words_list = rtf_df["Fake"].tolist()

# Populate dictionary
real_to_fake_dict = {}
for i in range(len(real_words_list)):
    real_to_fake_dict[real_words_list[i]] = fake_words_list[i]

## Get Definitions for all of the WinoDict Words
Get each of the definitions for the words.

In [7]:
from nltk.corpus import wordnet as wn

# Keep track of all the final definitions
final_definitions = []

# Loop through each real word and append
for word in real_words_list:
    definition = ""
    for synset in wn.synsets(word):
        definition += synset.definition() + ". "
    final_definitions.append(definition)

# Quick sanity check
assert(len(real_words_list) == len(final_definitions))
assert(len(fake_words_list) == len(final_definitions))

## Define Prediction and Actual Model
Retrieve G2G and R2G and baseline GPT-2 model

In [8]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# G2G/R2G used to Predict
predict_model = GPT2LMHeadModel.from_pretrained("weights/G2GNext1").to("cuda")
predict_tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
predict_tokenizer.add_tokens(['[CLS]'])

# GPT-2 Model and Tokenizer to be fine-tuned
model = GPT2LMHeadModel.from_pretrained('gpt2-medium').to("cuda")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')

# Standalone tokenizer
ORIG_TOKENIZER = GPT2Tokenizer.from_pretrained('gpt2-medium')

## Batch All Embeddings
Predict all of the embeddings for each of the definitions.

In [9]:
# Helpful Debug Message
print("Number of total definitions: " + str(len(final_definitions)))

# Tokenizing all of the definitions at once
predict_tokenizer.pad_token = tokenizer.eos_token
tokenized_inputs = predict_tokenizer(final_definitions, return_tensors="pt", padding='max_length', truncation=True, max_length=511)
tokenized_cls = predict_tokenizer([" [CLS]"] * len(final_definitions), return_tensors="pt")
tokenized_inputs['input_ids'] = torch.cat((tokenized_inputs['input_ids'], tokenized_cls['input_ids']), dim=1).to("cuda")

# Add the new tokens and resize the model embeddings matrix
displacement = len(tokenizer)
tokenizer.add_tokens(fake_words_list)
model.resize_token_embeddings(len(tokenizer))
params = model.state_dict()

# Adding new embeddings in a range of 4
for i in range(0, len(final_definitions), 4):
    outputs = predict_model(input_ids=tokenized_inputs['input_ids'][i:min(len(final_definitions), i + 4)], output_hidden_states=True)
    params['transformer.wte.weight'][displacement + i: displacement + min(len(final_definitions), i + 4),:] = outputs.hidden_states[-1][:,511,:].detach().clone()
model.load_state_dict(params)

Number of total definitions: 343


<All keys matched successfully>

## Function for Retrieving Fine-Tuned Model
Use this to retrieve the fine-tuned model.

In [2]:
def other_model():
    from transformers import AutoTokenizer, GPT2LMHeadModel
    tokenizer = AutoTokenizer.from_pretrained("weights/GWithFineTunedSentencesG2GInitializationT")
    model = GPT2LMHeadModel.from_pretrained("weights/GWithFineTunedSentencesG2GInitialization").to("cuda")

In [3]:
import pandas as pd
first_set = pd.read_csv("winodict/prob1_of_5.csv")
real_words_list = first_set['lemma'].tolist()
fake_words_list = first_set["fake_lemma"].tolist()
real_to_fake_dict = {}
for i in range(len(real_words_list)):
    if real_words_list[i] in real_to_fake_dict:
        real_to_fake_dict[real_words_list[i]].append(fake_words_list[i])
    else:
        real_to_fake_dict[real_words_list[i]] = [fake_words_list[i]]

In [4]:
import pandas
cbt_df = pd.read_csv("updated_cbt_extract/updated_cbt_info.csv")

In [5]:
with open("cbt_eval_info/ADVERSARIAL_indices", "r") as f:
    adversarial_indices = f.read()

adversarial_indices = adversarial_indices[1:len(adversarial_indices)-1].split(", ")
for i in range(len(adversarial_indices)):
    adversarial_indices[i] = int(adversarial_indices[i])

In [6]:
def evaluate_cbt(example):
    # extract the sentence
    sentence = example["sentences"]
    sentence = sentence.replace('"','')
    sentence = sentence.replace("\n","")
    sentence = sentence[1:len(sentence)-1]

    # extract the question
    question = example["question"]

    # make the total query with the sentence and the question
    total_query = sentence + " " + question

    
    #replace words in example with winodict fake words (currently, we replace all possible identified examples)
    replace_words = example["winodict_words"]
    replace_words = replace_words[1:len(replace_words)-1].replace("'","").split(",")

    for word in replace_words:
        word = word.strip()
        word = " "+word+" "
        while word in total_query:
            total_query = total_query.replace(word," "+real_to_fake_dict[word.strip()][0]+" ")
    

    # get the answer options for the model into a list
    options = example["options"].replace("'","")
    options = options.replace("\n","")
    options = options[1:len(options)-1].split(" ")

    # get the answer to the problem
    answer = example["answer"]

    
    if answer in replace_words:
        answer = real_to_fake_dict[answer][0]
    
        

    # initialize best answer and best loss (will be lowest value)
    best_answer = ""
    best_loss = float("inf")

    # compute the loss for each option (using full scoring)
    for option in options:
        
        if option in replace_words:
            option = real_to_fake_dict[option][0]
        

        updated_query = total_query.replace("XXXXX", option)
        
        # Tokenize each string and produce labels
        updated_input = tokenizer(updated_query, return_tensors="pt").to("cuda")

        if len(updated_input[0]) > 1024:
            return -1

        current_loss = model(**updated_input, labels=updated_input["input_ids"].to("cuda")).loss
        
        if current_loss < best_loss:
            best_answer = option
            best_loss = current_loss
    return best_answer == answer 

In [7]:
correct_replaced_list = []
total_replaced_list = []

correct = 0 
total = 0 
for index, row in cbt_df.iterrows():
    print(index)

    if index not in adversarial_indices:
        continue

    update = evaluate_cbt(row)
    if update == -1:
        continue
    if update == 1:
        correct_replaced_list.append(index)
    total_replaced_list.append(index)
    correct += update
    total += 1

correct_replaced_list = str(correct_replaced_list)
total_replaced_list = str(total_replaced_list)

print("")
print("Number correct: ", correct)
print("Total: ", total)
print("Correct percentage: ",correct/total)

information_list = str([correct, total, correct/total])

0


RuntimeError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 22.20 GiB total capacity; 8.03 GiB already allocated; 14.06 MiB free; 8.54 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF